In [22]:
import pandas as pd
import time
import gc
import sklearn
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.sparse import hstack
from scipy.sparse import coo_matrix
import xgboost as xgb
from scipy.stats import uniform
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import uniform
import ast
from sklearn.metrics import accuracy_score
random_state = 42
np.random.seed(random_state)

In [23]:
datosreducido = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/data_reducido.csv')  #  dataset
print("Tamaño del conjunto ", datosreducido.shape)

data50k = datosreducido.sample(n=50000, random_state=42)
print("Tamaño del conjunto ", data50k.shape)

validation_data_for_submit = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/validation_data_processed_for_datos_reducido.csv')
print("Tamaño del conjunto ", validation_data_for_submit.shape)


Tamaño del conjunto  (864459, 34)
Tamaño del conjunto  (50000, 34)
Tamaño del conjunto  (1139639, 33)


In [24]:
def apply_one_hot_encoding(df, columns):
    print("Aplicando One-Hot Encoding...")
    df_encoded = df.copy()
    for col in columns:
        if col in df_encoded.columns:
            # Asegurarse de que la columna esté en formato string
            df_encoded[col] = df_encoded[col].astype(str)
            
            print(f"Codificando columna: {col}")
            df_encoded = pd.get_dummies(df_encoded, columns=[col], sparse=True, dummy_na=False, dtype=int)
            print(f"Forma después de codificar {col}: {df_encoded.shape}")
        else:
            print(f"La columna '{col}' no está en el DataFrame")
    return df_encoded

def convert_string_to_list(df, list_columns):
    for col in list_columns:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        else:
            print(f"La columna '{col}' no está en el DataFrame")
    return df

def apply_multilabel_ohe(df, list_columns):
    df = convert_string_to_list(df, list_columns)
    df_encoded = df.copy()
    
    for col in list_columns:
        if col in df_encoded.columns:
            unique_values = set()
            df_encoded[col].dropna().apply(lambda x: unique_values.update(map(str, x)) if isinstance(x, list) else None)
            
            # Crear un DataFrame de OHE para esta columna
            ohe_df = pd.DataFrame()
            for value in unique_values:
                ohe_df[value] = df_encoded[col].apply(lambda x: 1 if isinstance(x, list) and str(value) in map(str, x) else 0)
            
            # Concatenar con el DataFrame original
            df_encoded = pd.concat([df_encoded, ohe_df], axis=1)
            
            # Eliminar la columna original con listas si ya no es necesaria
            df_encoded = df_encoded.drop(columns=[col])
        else:
            print(f"La columna '{col}' no está en el DataFrame")

    return df_encoded

columns_to_encode = [
    'action_categorical_0', 'action_categorical_1', 'action_categorical_2', 
    'action_categorical_3', 'action_categorical_4',
    'auction_categorical_0', 'auction_categorical_11', 'auction_categorical_12', 
    'auction_categorical_3', 'auction_categorical_7', 'auction_categorical_8', 
    'auction_categorical_9', 'creative_categorical_0', 
    'creative_categorical_11', 'creative_categorical_12', 'creative_categorical_2', 
    'creative_categorical_4', 'creative_categorical_5', 'creative_categorical_6', 
    'creative_categorical_7', 'creative_categorical_8', 'creative_categorical_9', 
    'device_id_type'
]

# Columnas con listas para OHE
list_columns = ['action_list_1', 'action_list_2', 'auction_list_0']

# Aplicar codificación one-hot a las columnas categóricas
data_OHE_Cat = apply_one_hot_encoding(data50k, columns_to_encode)

# Aplicar OHE a las columnas de listas
data_OHE_Cat_List = apply_multilabel_ohe(data_OHE_Cat, list_columns)
#Valid_OHE_Cat_List = apply_multilabel_ohe(Valid_OHE_Cat, list_columns)

print("DataFrame después del MultiLabel OHE:")
print(f"Forma después de codificar train: {data_OHE_Cat.shape}")

Aplicando One-Hot Encoding...
Codificando columna: action_categorical_0
Forma después de codificar action_categorical_0: (50000, 42)
Codificando columna: action_categorical_1
Forma después de codificar action_categorical_1: (50000, 55)
Codificando columna: action_categorical_2
Forma después de codificar action_categorical_2: (50000, 153)
Codificando columna: action_categorical_3
Forma después de codificar action_categorical_3: (50000, 169)
Codificando columna: action_categorical_4
Forma después de codificar action_categorical_4: (50000, 729)
Codificando columna: auction_categorical_0
Forma después de codificar auction_categorical_0: (50000, 1168)
Codificando columna: auction_categorical_11
Forma después de codificar auction_categorical_11: (50000, 5144)
Codificando columna: auction_categorical_12
Forma después de codificar auction_categorical_12: (50000, 6912)
Codificando columna: auction_categorical_3
Forma después de codificar auction_categorical_3: (50000, 42697)
Codificando columna

/var/folders/m6/yrlm4ckj0r71yq2s7_1v54kh0000gn/T/ipykernel_51892/3983968912.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ohe_df[value] = df_encoded[col].apply(lambda x: 1 if isinstance(x, list) and str(value) in map(str, x) else 0)
/var/folders/m6/yrlm4ckj0r71yq2s7_1v54kh0000gn/T/ipykernel_51892/3983968912.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ohe_df[value] = df_encoded[col].apply(lambda x: 1 if isinstance(x, list) and str(value) in map(str, x) else 0)
/var/folders/m6/yrlm4ckj0r71yq2s7_1v54kh0000gn/T/ipyke

DataFrame después del MultiLabel OHE:
Forma después de codificar train: (50000, 48639)


In [25]:
y = data_OHE_Cat_List[['Label']].copy() # Usamos copy para no modificar un view abajo, ya que genera un warning.
y
unique_labels = data_OHE_Cat_List['Label'].unique()
print(f'Valores únicos en la columna Label: {unique_labels}')
y[y['Label'] == 0. ] = 0
y[y['Label'] == 1.] = 1
y['Label'] = y['Label'].fillna(0)
y['Label'] = y['Label'].astype(int)
X = data_OHE_Cat_List.drop('Label', axis = 1)
X

Valores únicos en la columna Label: [0 1]


,auction_age,auction_bidfloor,auction_time,creative_height,creative_width,has_video,timezone_offset,action_categorical_0_11b7af3d,action_categorical_0_604d011f,action_categorical_0_6b5513a4,...,IAB9-30,IAB7-2,IAB7-10,IAB10-3,IAB6-4,IAB19-3,IAB6-9,IAB18,IAB17-39,lifestyle
666392,NaN,0.020000,1.516143e+09,50.0,320.0,False,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
675483,NaN,0.310000,1.516539e+09,250.0,300.0,False,4.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
381289,NaN,0.070000,1.516390e+09,250.0,300.0,False,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
174376,NaN,0.559000,1.516374e+09,50.0,320.0,False,1.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
220432,31.0,0.020000,1.516380e+09,250.0,300.0,False,4.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631126,NaN,0.100000,1.516479e+09,50.0,320.0,False,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72307,39.0,0.850000,1.516206e+09,NaN,NaN,False,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
90504,16.0,0.020000,1.516473e+09,50.0,320.0,False,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
521083,NaN,0.307692,1.516557e+09,50.0,320.0,False,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y,
                                                  train_size=0.7,
                                                  random_state=42,
                                                  stratify=y)
print("ready")

# Dividir el resto en validación y prueba
X_val, X_test, Y_val, y_test = train_test_split(X_tmp, y_tmp,
                                                train_size=0.5,
                                                random_state=42,
                                                stratify=y_tmp)

print(f'Cantidad de datos de entrenamiento: {len(X_train)}')
print(f'Cantidad de datos de validación: {len(X_val)}')
print(f'Cantidad de datos de prueba: {len(X_test)}')

val_test_size = 0.3 # Proporción de la suma del test de validación y del de test.
X_train, X_tmp, Y_train, Y_tmp = train_test_split(X, y,
                                                  train_size = 0.7,
                                                  random_state = 42,
                                                  stratify = y)

X_val, X_test, Y_val, y_test = train_test_split(X_tmp, y_tmp,
                                                train_size=0.5,
                                                random_state= 42,
                                                stratify=y_tmp)
print(f'Cantidad de datos de train: {len(X_train)}')
print(f'Cantidad de datos de validación: {len(X_val)}')
print(f'Cantidad de datos de test: {len(X_test)}')

ready
Cantidad de datos de entrenamiento: 35000
Cantidad de datos de validación: 7500
Cantidad de datos de prueba: 7500
Cantidad de datos de train: 35000
Cantidad de datos de validación: 7500
Cantidad de datos de test: 7500


In [27]:
print(X_train.dtypes.head(900))

auction_age                                float64
auction_bidfloor                           float64
auction_time                               float64
creative_height                            float64
creative_width                             float64
                                        ...       
auction_categorical_0_6e069bec    Sparse[int64, 0]
auction_categorical_0_6e74066f    Sparse[int64, 0]
auction_categorical_0_6e9ad7fd    Sparse[int64, 0]
auction_categorical_0_6eba7824    Sparse[int64, 0]
auction_categorical_0_707fbf0e    Sparse[int64, 0]
Length: 900, dtype: object


In [28]:
# Define the XGBoost classifier
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
                            seed=42,
                            eval_metric='auc',  # You can also use 'logloss' for binary classification
                            use_label_encoder=False)

In [29]:
import scipy.sparse as sp

X_train_sparse = sp.csr_matrix(X_train)
X_val_sparse = sp.csr_matrix(X_val)
X_test_sparse = sp.csr_matrix(X_test)

# Ahora puedes usar XGBoost con matrices sparse
clf_xgb.fit(X_train_sparse, y_train, eval_set=[(X_val_sparse, Y_val)], verbose=True)

/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [14:39:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.75379
[1]	validation_0-auc:0.75646
[2]	validation_0-auc:0.77839
[3]	validation_0-auc:0.78123
[4]	validation_0-auc:0.78308
[5]	validation_0-auc:0.78306
[6]	validation_0-auc:0.78143
[7]	validation_0-auc:0.79876
[8]	validation_0-auc:0.80058
[9]	validation_0-auc:0.79576
[10]	validation_0-auc:0.79668
[11]	validation_0-auc:0.79923
[12]	validation_0-auc:0.79970
[13]	validation_0-auc:0.80074
[14]	validation_0-auc:0.79801
[15]	validation_0-auc:0.79550
[16]	validation_0-auc:0.79689
[17]	validation_0-auc:0.79732
[18]	validation_0-auc:0.79828
[19]	validation_0-auc:0.79928
[20]	validation_0-auc:0.80062
[21]	validation_0-auc:0.80168
[22]	validation_0-auc:0.79984
[23]	validation_0-auc:0.80029
[24]	validation_0-auc:0.80053
[25]	validation_0-auc:0.80152
[26]	validation_0-auc:0.80294
[27]	validation_0-auc:0.80248
[28]	validation_0-auc:0.80150
[29]	validation_0-auc:0.79903
[30]	validation_0-auc:0.79959
[31]	validation_0-auc:0.80020
[32]	validation_0-auc:0.80016
[33]	validation_0-au

KeyboardInterrupt: 

In [20]:
import pandas as pd
import gc
from sklearn.impute import SimpleImputer
import scipy.sparse as sp

# Cargar los datos de evaluación (test)
eval_data = pd.read_csv("/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_test.csv")


# Procesar los datos de test (seleccionando solo las columnas numéricas, como en el entrenamiento)
eval_data_num = eval_data.select_dtypes(include='number')

# Imputar valores faltantes, si es necesario (usa la misma estrategia que en entrenamiento)
imputer = SimpleImputer(strategy="mean")
eval_data_imputed = imputer.fit_transform(eval_data_num.drop(columns=["id"]))

# Convertir los datos a formato sparse para ser usados en el modelo XGBoost
eval_data_sparse = sp.csr_matrix(eval_data_imputed)

# Hacer predicciones con el modelo XGBoost entrenado
y_preds = clf_xgb.predict_proba(eval_data_sparse)[:, 1]  # Obtener la probabilidad de la clase positiva (1)

# Crear el DataFrame para el envío a Kaggle
submission_df = pd.DataFrame({
    "id": eval_data["id"],
    "Label": y_preds
})

# Convertir el ID a entero (según el formato requerido por Kaggle)
submission_df["id"] = submission_df["id"].astype(int)

# Guardar el archivo CSV para enviar a Kaggle
submission_df.to_csv("xgboost_model_submission.csv", sep=",", index=False)

print("Archivo de predicción creado: xgboost_model_submission.csv")

print("Archivo de predicción creado: xgboost_model_submission.csv")

NotFittedError: need to call fit or load_model beforehand